<a href="https://colab.research.google.com/github/grzech903/Raster-Clipping-and-reclassification/blob/main/forest_clipper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install rasterio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.6/20.6 MB 94.2 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import rasterio
from rasterio.mask import mask
from rasterio.windows import Window
import numpy as np
from tqdm import tqdm
import geopandas as gpd
from shapely.geometry import box, mapping

In [ ]:


# reclassification classes
reclass_values = [
    (0, 0),    # Class 1: 0-0
    (1, 15),   # Class 2: 1-15
    (16, 30),  # Class 3: 16-30
    (30, 60),  # Class 4: 30-60
    (61, 999)  # Class 5: Anything above 60
]

# reclassification
raster_path = '/content/drive/MyDrive/colab/Forest_height_2019_AUS.tif'
outdir = '/content/drive/MyDrive/colab/aus'

def reclassify_raster(input_path, output_path, reclass_values, block_size=512):
    with rasterio.open(input_path) as src:
        profile = src.profile


        with rasterio.open(output_path, 'w', **profile) as dst:
            width = src.width
            height = src.height


            for i in tqdm(range(0, height, block_size), desc="Processing"):
                for j in range(0, width, block_size):
                    window = Window(j, i, min(block_size, width - j), min(block_size, height - i))

                    block = src.read(1, window=window, masked=True)

                    for index, reclass in enumerate(reclass_values):
                        lower_bound, upper_bound = reclass
                        mask = (block >= lower_bound) & (block <= upper_bound)
                        block = np.where(mask, index + 1, block)

                    dst.write(block, 1, window=window)

    print(f"Reclassification completed. Result saved to: {output_path}")

reclassify_raster(raster_path, os.path.join(outdir, 'ReclassifiedRaster.tif'), reclass_values, block_size=512)




Processing: 100%|██████████| 290/290 [09:53<00:00,  2.05s/it]


Reclassification completed. Result saved to: /content/drive/MyDrive/colab/aus/ReclassifiedRaster.tif


In [ ]:


# removal of unnecessary countries from the raster

raster_path = '/content/drive/MyDrive/colab/aus/ReclassifiedRaster.tif'
geojson_path = '/content/drive/MyDrive/colab/countries.geojson'
outdir = '/content/drive/MyDrive/colab/aus/limited_countries'

def limit_geojson_to_raster_extent(raster_path, geojson_path, outdir):
    os.makedirs(outdir, exist_ok=True)
    with rasterio.open(raster_path) as src:
        raster_bounds = src.bounds
    countries = gpd.read_file(geojson_path)

    raster_geom = box(*raster_bounds)

    countries_clipped = countries[countries.geometry.intersects(raster_geom)]
    countries_clipped = countries_clipped.assign(geometry=countries_clipped.intersection(raster_geom))

    clipped_geojson_path = os.path.join(outdir, 'clipped_countries.geojson')
    countries_clipped.to_file(clipped_geojson_path, driver='GeoJSON')

    print(f"Clipped GeoJSON saved to: {clipped_geojson_path}")


limit_geojson_to_raster_extent(raster_path, geojson_path, outdir)

Clipped GeoJSON saved to: /content/drive/MyDrive/colab/aus/limited_countries/clipped_countries.geojson


In [ ]:


# Preparation of polygons of individual countries

geojson_path = '/content/drive/MyDrive/colab/aus/limited_countries/clipped_countries.geojson'
outdir = '/content/drive/MyDrive/colab/aus/countrynam'

gdf = gpd.read_file(geojson_path)
for index, country in gdf.iterrows():
    country_name = country['ADMIN']


    country_name_no_space = country_name.replace(' ', '')

    country_outpath = os.path.join(outdir, f'{country_name_no_space}.geojson')


    single_country_gdf = gpd.GeoDataFrame(geometry=[country.geometry], crs=gdf.crs)


    single_country_gdf.to_file(country_outpath, driver='GeoJSON')

    print(f'Zapisano plik dla kraju: {country_name}')

Zapisano plik dla kraju: Ashmore and Cartier Islands
Zapisano plik dla kraju: Australia
Zapisano plik dla kraju: Coral Sea Islands
Zapisano plik dla kraju: Fiji
Zapisano plik dla kraju: New Caledonia
Zapisano plik dla kraju: Norfolk Island
Zapisano plik dla kraju: New Zealand
Zapisano plik dla kraju: Papua New Guinea
Zapisano plik dla kraju: Solomon Islands
Zapisano plik dla kraju: Vanuatu


In [ ]:



# Cliping the raster to country size.
# I am omitting here large geojesons with a large number of vertrexes because I do not have enough memory to cut them using this method. Big geojeson i extrack manualy with  GDAL in qgis (I couldn't get the gdal to work in colaba )

def list_large_geojson(geojson_folder, max_size=100000):
    large_files = []
    for file in os.listdir(geojson_folder):
        if file.endswith('.geojson'):
            file_path = os.path.join(geojson_folder, file)
            file_size = os.path.getsize(file_path)
            if file_size > max_size:
                large_files.append((file, file_size))
    return large_files


polygons_folder = '/content/drive/MyDrive/colab/aus/countrynam'
raster_path = '/content/drive/MyDrive/colab/aus/ReclassifiedRaster.tif'
outdir = '/content/drive/MyDrive/colab/aus/country_rasters'


polygon_files = [os.path.join(polygons_folder, file) for file in os.listdir(polygons_folder) if file.endswith('.geojson')]
large_geojson_files = list_large_geojson(polygons_folder)


with rasterio.open(raster_path) as src:
    for polygon_file in tqdm(polygon_files, desc='Processing Countries', unit='country', leave=False):
        if polygon_file in [os.path.join(polygons_folder, file[0]) for file in large_geojson_files]:
            tqdm.write(f"Pomiń plik: {os.path.basename(polygon_file)} (większy niż 100 KB)")
            continue

        country_name = os.path.splitext(os.path.basename(polygon_file))[0]
        country_gdf = gpd.read_file(polygon_file)

        for index, country_polygon in country_gdf.iterrows():
            try:
                out_image, out_transform = mask(src, [country_polygon.geometry], crop=True)


                out_meta = src.meta.copy()
                out_meta.update({
                    "driver": "GTiff",
                    "height": out_image.shape[1],
                    "width": out_image.shape[2],
                    "transform": out_transform
                })

                country_outpath = os.path.join(outdir, f'{country_name}_{index}.tif')

                with rasterio.open(country_outpath, "w", **out_meta) as dest:
                    dest.write(out_image)

                tqdm.write(f'Zapisano plik dla kraju: {country_name}_{index}.tif')

            finally:
                out_image = None
                out_transform = None
                out_meta = None


Processing Countries:   0%|          | 0/10 [00:00<?, ?country/s]

Zapisano plik dla kraju: AshmoreandCartierIslands_0.tif
Pomiń plik: Australia.geojson (większy niż 100 KB)
Zapisano plik dla kraju: CoralSeaIslands_0.tif


Processing Countries:  40%|████      | 4/10 [00:23<00:34,  5.75s/country]

Zapisano plik dla kraju: Fiji_0.tif


Processing Countries:  60%|██████    | 6/10 [00:30<00:17,  4.44s/country]

...

Processing Countries:  80%|████████  | 8/10 [00:30<00:05,  2.69s/country]

Zapisano plik dla kraju: PapuaNewGuinea_0.tif


Processing Countries:  90%|█████████ | 9/10 [00:33<00:02,  2.69s/country]

Zapisano plik dla kraju: SolomonIslands_0.tif


Zapisano plik dla kraju: Vanuatu_0.tif


In [ ]:
#List of countries to be done manually

input_geojson_folder = '/content/drive/MyDrive/colab/nam/countrynam'

def list_large_geojson_files(folder_path, size_threshold_kb=100):
    large_files = []

    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)
        if filename.endswith('.geojson') and os.path.getsize(file_path) / 1024 > size_threshold_kb:
            large_files.append(filename)

    return large_files

large_geojson_files = list_large_geojson_files(input_geojson_folder)

print("Pliki GeoJSON powyżej 100 KB:")
for file in large_geojson_files:
    print(file)

Pliki GeoJSON powyżej 100 KB:
Canada.geojson
Cuba.geojson
Mexico.geojson
UnitedStatesofAmerica.geojson
big.geojson
